![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [119]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.linear_model import Lasso  # Import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# check the data
df = pd.read_csv("rental_info.csv")
print(df.head())

# Preprocess
df.isna().sum() # No na

# add rental_length_days column
df["rental_length_days"] = (pd.to_datetime(df["return_date"]) - pd.to_datetime(df["rental_date"])).dt.days
df["rental_length_days"]

# create dummy variables
# Add dummy for deleted scenes
df["deleted_scenes"] =  np.where(df["special_features"].str.contains("Deleted Scenes"), 1, 0)
# Add dummy for behind the scenes
df["behind_the_scenes"] =  np.where(df["special_features"].str.contains("Behind the Scenes"), 1, 0)
df.head()

# split the data
X = df.drop(["special_features","rental_length_days", "rental_date", "return_date"], axis=1)

y = df["rental_length_days"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Trying to find the best features, while I am not sure what the most important features in movies case
lasso = Lasso(alpha=0.25, random_state=9)
lasso.fit(X_train, y_train)

# get the feature names
feature_names = X_train.columns

# get the coefficients
coefficients = lasso.coef_

# create a DataFrame to view both features and their corresponding coefficients
features_df = pd.DataFrame(list(zip(feature_names, coefficients[coefficients>0])), columns=["Feature", "Coefficient"])
print(features_df)

# Linear regression
X_lasso_train, X_lasso_test = X_train.iloc[:, coefficients > 0], X_test.iloc[:, coefficients > 0]
lr = LinearRegression()
lr = lr.fit(X_lasso_train, y_train)
y_test_pred = lr.predict(X_lasso_test)
mse_lr_lasso = MSE(y_test, y_test_pred)
print(mse_lr_lasso) # >than the required


# GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=9)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
mse = MSE(y_test, y_pred)
print(mse)

best_model = gbr
best_mse = mse

                 rental_date  ... rental_rate_2
0  2005-05-25 02:54:33+00:00  ...        8.9401
1  2005-06-15 23:19:16+00:00  ...        8.9401
2  2005-07-10 04:27:45+00:00  ...        8.9401
3  2005-07-31 12:06:41+00:00  ...        8.9401
4  2005-08-19 12:30:04+00:00  ...        8.9401

[5 rows x 15 columns]
        Feature  Coefficient
0        amount     0.711035
1  release_year     0.033076
2   rental_rate     0.000003
4.812297241276244
2.4253464800253557
